# retriever

> Functionalities to Retrieve the publications

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
import pandas as pd
from datetime import datetime, date, timedelta
from fastcore.all import *
from pubmed_lib.data import *
from pubmed_lib.parser import *
# from pubmed_lib.core import *
from pubmed_lib.viz import *

In [ ]:
#| hide
pd.set_option('display.max_columns',30)
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_rows',500)

In [ ]:
#| hide
def getParsedArticles(query, #Query to Pubmed, by default searching Author, give a valid 'search_tag' to search by different tag
                      f_year = datetime.now().year, #The max year to search papers from, default CY
                      years = 3, #Window of years back to search papers
                      **kwargs
                     ):
    cy = f_year
    my = cy - years
    results = searchpb(query, mindate=my, maxdate=cy, **kwargs)
    id_list = results['IdList']
    if len(id_list) == 0:
        return 0
    papers = fetch_details(id_list)
    n_papers = len(id_list)
    print('checking in {} Articles'.format(n_papers))
    articles=[]
    for i, paperinfo in enumerate(papers['PubmedArticle']):
        article = parse_paperinfo(paperinfo)
        if int(article['published']) < my:
            continue
        articles.append(article)
    print(f'Keeping with {len(articles)} from last {years} years')
    return articles

In [ ]:
#| hide
def getParsedArticlesPeriod(query, #Query to Pubmed, by default searching Author, give a valid 'search_tag' to search by different tag
                            maxdate=datetime.now().year, #The max year to search papers from, default CY
                            years = 3, #Window of years back to search papers
                            top_n=None, #Number of papers to retreive, ordered by published date
                            **kwargs):
    query = query + '[Author]'
    results = searchpb(query, 1000, maxdate = maxdate, mindate = maxdate - years, **kwargs)
    id_list = results['IdList']
    if len(id_list) == 0:
        return ([],0)
    papers = fetch_details(id_list)
    n_papers = len(id_list)
    if verbose:
        print('checking in {} Articles'.format(n_papers))
    articles=[]
    for i, paperinfo in enumerate(papers['PubmedArticle']):
        article = parse_paperinfo(paperinfo)
        if int(article['published']) < maxdate - years :
            # print('to old, article published on {}'.format(article['published']))
            continue
        articles.append(article)
    if len(articles) == 0:
        if verbose:
            print('No articles in the time period')
        return ([],n_papers)
    elif top_n:
        df = pd.DataFrame(articles).sort_values('published', ascending=False)
        df = df.iloc[:top_n]
        articles = df.to_dict('records')
    if verbose:
        print('Keeping with {} from last {} years'.format(len(articles), years))
    return (articles, n_papers)

In [ ]:
#| hide
def fetchPubmedArticles(query, start, end, path, db_path = '/Volumes/Users/matu/Documents/Xcode/SFDC/db.pckl'):
    """Function to search in pubmed by query, start and end year.
    It checks first in the database of abstracts downloaded before.
    Create a csv file with the parsed pubmed results including abstract, authors, etc. (look at pubmed_utils)

    return (pd.Dataframe) -> the DataFrame with all the information retrieved"""
    db = loadDB(db_path)
    if query not in db:
        print('adding new year {} for {}'.format(start, query))
        (pubmedData, total) = getParsedArticlesPeriod(query, start, end)
        if pubmedData == 0:
            db.update({query: {str(start):[total, 0]}})
            return 
        else:
            db.update({query: {str(start):[total, len(pubmedData)]}})
    else:
        if (str(start) in db[query]) and (str(end) in db[query]):
            print(f"{query} already in DB with year {start} - {end}, passing")
            df = pd.read_csv('{}/{}_{}_{}.csv'.format(path, query, start, start - end))
            return df
        else:
            (pubmedData, total) = getParsedArticlesPeriod(query, start, end)
            if pubmedData == 0:
                db[query].update({str(start):[total, 0]})
                return 
            else:
                db[query].update({str(start):[total, len(pubmedData)]})
    df = pd.DataFrame(pubmedData)
    file_output = '{}/{}_{}_{}.csv'.format(path, query, start, start - end)
    df.to_csv(file_output)
    saveDB(db, db_path)
    if df.shape[0] >= 10:
        df = df.sort_values('published', ascending=False)
        print('Using the 10 newer papers')
        return df.iloc[:10]
    return df

In [ ]:
#| hide
def retrieveArticles():
    results = searchpb('Peter Ihnat[Author]')
    papers = fetch_details(results['IdList'])
    articles=[]
    for i, paperinfo in enumerate(papers['PubmedArticle']):
        article = parse_paperinfo(paperinfo)
        articles.append(article)
    return articles

In [ ]:
#| hide
def get_email(query: str, **kwargs):
    results = searchpb(query, **kwargs)
    id_list = results['IdList']
    summary = []
    if len(id_list) == 0:
        print('No existen articulos nuevos para esta consulta')
        return
    print('Existen ' + str(len(id_list)) + ' articulos')
    search_details = fetch_details(id_list)
    papers = search_details['PubmedArticle']
    for paperinfo in papers:
        article = parse_paperinfo(paperinfo)
        summary.append(article)
    pd_summary = pd.DataFrame(summary)
###    pd_summary.to_excel(query.replace(' ','_') + '.xlsx')
    pd_det = pd_summary.explode('autorlist')
    pd_det = pd_det.reset_index(drop=True)
    pd_det = pd.concat([pd_det['autorlist'].apply(pd.Series), pd_det.drop(['autorlist'], axis=1)], axis=1)
    # pd_det.to_excel(query.replace(' ','_') + '_det.xlsx')
    plot_countries(pd_det)
    plot_timeline(pd_summary)
#     df_filt =pipeline(pd_det, region)
#     net, df = pd2ng(df_filt)
#     net, df = measure_network(net, df)
#     top50 = df.sort_values('paper',ascending=False).index[:50]
#     # plot_circos(top50, net, table_name)
#     plot_histogram(df, table_name)
#     # pd_det2 = pd_summary.explode('autorlist')
#     pd_det2 = pd_det[['Fname','Lname','emails','affiliations','countries','state',
#                       'identifier','name','title','doi','pubmed','published']]
#     df_cutoff = df.join(pd_det2.set_index('name'))
#     writer_cutoff = pd.ExcelWriter(table_name + '_cut' + '.xlsx', engine='xlsxwriter')
#     df_cutoff = df_cutoff.sort_values('paper', ascending=False)
#     df_cutoff.to_excel(writer_cutoff)
#     writer_cutoff.close()
#     if aff:
#         return pd_det2.loc[(pd_det2.emails != '')].sort_values('published',ascending=False)
#     elif all_res:
#         return pd_det2.loc[(pd_det2.emails != '')].drop_duplicates(['Fname', 'Lname','emails']).sort_values('published',ascending=False)
                    
#     return pd_det2.loc[(pd_det2.emails != '') & (pd_det2.Lname.str.contains(query.split(' ')[-1]))]#sort_values('published',ascending=False)
    return pd_det

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()